# Scale effects

In [1]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
import numpy as np
import yaml

from vessel_manoeuvring_models.visualization.plot import track_plot, plot
from phd.visualization.plot_ship import track_plots
from wPCC_pipeline.pipelines.preprocess.nodes import load, add_thrust
from sklearn.metrics import mean_squared_error
import sympy as sp
from functools import reduce
from operator import add
from vessel_manoeuvring_models.symbols import *
import vessel_manoeuvring_models.symbols as symbols
from IPython.display import display
from vessel_manoeuvring_models.prime_system import df_prime
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify, get_function_subs
from vessel_manoeuvring_models.parameters import df_parameters
from phd.helpers import mean
from vessel_manoeuvring_models.angles import mean_angle
from vessel_manoeuvring_models.angles import smallest_signed_angle
import inspect
from phd.visualization.plot_prediction import plot_total_force, plot_force_components
from vessel_manoeuvring_models.models.modular_simulator import ModularVesselSimulator, function_eq, subs_simpler
from scipy.optimize import least_squares
from vessel_manoeuvring_models.models.diff_eq_to_matrix import DiffEqToMatrix
from vessel_manoeuvring_models.prime_system import PrimeSystem
import statsmodels.api as sm
import logging
log = logging.getLogger(__name__)

import matplotlib.pyplot as plt
plt.style.use('bmh')

In [3]:
%reload_kedro
ship='7m'
model_loaders = catalog.load(f"{ship}.models")
time_series = catalog.load(f"{ship}.tests")
#time_series = catalog.load(f"{ship}.tests_ek_smooth")
time_series_meta_data = catalog.load(f"{ ship }.test_meta_data")
time_series_meta_data['date'] = pd.to_datetime(time_series_meta_data['date'])
time_series_meta_data.sort_values(by='date', inplace=True)
time_series_meta_data.index = time_series_meta_data.index.astype(str)
ship_data = catalog.load(f"{ship}.ship_data")
units = catalog.load(f"{ship}.units")

[07/18/23 16:06:02] INFO     Resolved project path as: /home/maa/dev/PHD.                           ]8;id=107082;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=11296;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py#134\134]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[07/18/23 16:06:12] INFO     Kedro project phd                                                      ]8;id=484418;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=895267;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py#103\103]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=469051;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=754212;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py#104\104]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=30743;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=435816;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py#110\110]8;;\

                    INFO     Loading data from '7m.models' (PartitionedDataSet)...              ]8;id=73719;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=166361;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from '7m.tests' (PartitionedDataSet)...               ]8;id=7222;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=311563;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from '7m.test_meta_data' (CSVDataSet)...              ]8;id=250888;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=816302;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from '7m.ship_data' (YAMLDataSet)...                  ]8;id=425588;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=317086;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from '7m.units' (YAMLDataSet)...                      ]8;id=373321;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=914325;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [4]:
%reload_kedro
ship='wPCC'
time_series_wPCC = catalog.load(f"{ship}.time_series_preprocessed.ek_smooth")
time_series_meta_data_wPCC = catalog.load(f"{ ship }.time_series_meta_data")
ship_data_wPCC = catalog.load(f"{ship}.ship_data")
model_loaders_MDL = catalog.load(f"{ship}.models")

                    INFO     Resolved project path as: /home/maa/dev/PHD.                           ]8;id=605033;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=667954;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py#134\134]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[07/18/23 16:06:22] INFO     Kedro project phd                                                      ]8;id=238129;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=755523;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py#103\103]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=931451;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=657120;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py#104\104]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=744347;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=900919;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py#110\110]8;;\

                    INFO     Loading data from 'wPCC.time_series_preprocessed.ek_smooth'        ]8;id=980148;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=44892;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             (PartitionedDataSet)...                                                               

                    INFO     Loading data from 'wPCC.time_series_meta_data' (CSVDataSet)...     ]8;id=167569;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=682746;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'wPCC.ship_data' (YAMLDataSet)...                ]8;id=659923;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=431103;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'wPCC.models' (PartitionedDataSet)...            ]8;id=944226;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=614396;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [5]:
def preprocess(data):
    data['-delta_deg'] = -np.rad2deg(data['delta'])
    psi0 = data.iloc[0]['psi'].copy()
    data['psi']-=psi0
    data['x0']-=data['x0'].iloc[0]
    data['y0']-=data['y0'].iloc[0]
    x0 = data['x0'].copy()
    y0 = data['y0'].copy()
    data['x0'] = x0*np.cos(psi0) + y0*np.sin(psi0)
    data['y0'] = -x0*np.sin(psi0) + y0*np.cos(psi0)
    data['psi_deg'] = np.rad2deg(smallest_signed_angle(data['psi']))

## Field study data

In [6]:
field_data = {}

id=6
data = time_series[str(id)]().loc[0:30].copy()
preprocess(data)
field_data[id]=data

id=8
data = time_series[str(id)]().loc[0:30].copy()
preprocess(data)
field_data[id]=data



## Model test data

In [7]:
id = 22764
data_wPCC = time_series_wPCC[f'wpcc.updated.{id}.ek_smooth']().loc[0:data.index[-1]]
i = np.argmax(np.abs(np.gradient(data_wPCC['delta'],data_wPCC.index)) > np.deg2rad(2.32)*np.sqrt(ship_data_wPCC['scale_factor']))
data_wPCC=data_wPCC.iloc[i:].copy()
data_wPCC['x0']-=data_wPCC.iloc[0]['x0']
data_wPCC['y0']-=data_wPCC.iloc[0]['y0']
data_wPCC.index-=data_wPCC.index[0]

data_wPCC['V'] = data_wPCC['U'] = np.sqrt(data_wPCC['u']**2 + data_wPCC['v']**2)
data_wPCC['-delta_deg'] = -np.rad2deg(data_wPCC['delta'])
data_wPCC['psi']-=data_wPCC.iloc[0]['psi']
data_wPCC['psi_deg'] = np.rad2deg(data_wPCC['psi'])
data_wPCC["rev"] = data_wPCC[["Prop/SB/Rpm", "Prop/PS/Rpm"]].mean(axis=1)

In [8]:
model_MDL = model_loaders_MDL['vmm_martins_simple_thrust.MDL_inverse_dynamics']()
model = model_loaders['vmm_martins_simple_thrust.MDL_inverse_dynamics']()
ship_parameters = model.prime_system.unprime(model_MDL.ship_parameters_prime)
model.set_ship_parameters(ship_parameters)

In [9]:
ship_data

{'A_R': 0.07777777777777778,
 'A_XV': 1.1200000000000003,
 'A_YV': 6.4046533,
 'B': 1.3,
 'CB': 0.45034232324249973,
 'D': 0.16666666666666666,
 'I_z': 3430.04989355463,
 'L': 6.886666666666668,
 'T': 0.2833333333333332,
 'TWIN': 1.0,
 'b_R': 0.23933333333333334,
 'm': 1157.1836666666663,
 'n_prop': 2.0,
 'rho': 1013.0,
 'rho_A': 1.2409250000000003,
 'scale_factor': 30,
 'tdf': 0.117,
 'volume': 1.1423333333333332,
 'w_p0': 0.151,
 'x_G': -0.3294581333333334,
 'x_p': -3.326486749598252,
 'x_r': -3.4932213333333335}

In [10]:
ship_data['x_r']

-3.4932213333333335

In [11]:
ship_data_wPCC['x_r']*ship_data_wPCC['scale_factor']/ship_data['scale_factor']

-3.4432213333333337

In [12]:
model_MDL.ship_parameters['L']*model_MDL.ship_parameters['scale_factor']/ship_data['scale_factor']

6.886666666666668

In [13]:
model_MDL.ship_parameters

{'T': 0.2063106796116504,
 'L': 5.014563106796117,
 'CB': 0.45034232324249973,
 'B': 0.9466019417475728,
 'rho': 1000,
 'x_G': 0,
 'm': 441.0267843660858,
 'I_z': 693.124396594905,
 'volume': 0.4410267843660858,
 'scale_factor': 41.2,
 'x_r': -2.42219908951329,
 'TWIN': 1,
 'x_p': -2.42219908951329,
 'D': 0.12135922330097086,
 'w_p0': 0.151,
 'A_XV': 0.5938354227542653,
 'A_YV': 3.3958124999999995,
 'rho_A': 1.225,
 'n_prop': 2,
 'tdf': 0.117,
 'A_R': 0.041238571024601746,
 'b_R': 0.17427184466019416}

In [14]:
model.ship_parameters

{'T': 0.2879961277831557,
 'L': 7.0,
 'CB': 0.45034232324249973,
 'B': 1.3213939980638916,
 'rho': 1025.0,
 'x_G': 0.0,
 'm': 1229.6560411187577,
 'I_z': 3765.8216259261953,
 'volume': 1.1996644303597637,
 'scale_factor': 41.2,
 'x_r': -3.3812304811986102,
 'TWIN': 1.0,
 'x_p': -3.3812304811986102,
 'D': 0.16940948693126812,
 'w_p0': 0.151,
 'A_XV': 1.1571668342565615,
 'A_YV': 6.617189628278426,
 'rho_A': 1.2556250000000002,
 'n_prop': 2.0,
 'tdf': 0.117,
 'A_R': 0.08035880793448343,
 'b_R': 0.24327202323330105}

## Scaling model test data to 7m

In [15]:
ship_data_wPCC['L']*ship_data_wPCC['scale_factor']

206.60000000000002

In [16]:
ship_data['L']*ship_data['scale_factor']

206.60000000000002

In [17]:
ship_data['scale_factor']

30

In [18]:
ship_data_wPCC['B']*ship_data_wPCC['scale_factor']

39.00000000000001

In [19]:
ship_data['B']*ship_data['scale_factor']

39.0

In [20]:
ship_data_wPCC['volume']*ship_data_wPCC['scale_factor']**3

30842.999999999996

In [21]:
ship_data['volume']*ship_data['scale_factor']**3

30842.999999999996

In [22]:
1165*ship_data['scale_factor']**3

31455000

In [23]:
ship_data['scale_factor']

30

In [24]:
-ship_data_wPCC['L']/2

-2.5072815533980584

In [25]:
ship_data_wPCC['L']

5.014563106796117

In [26]:
ship_data_wPCC['x_r']

-2.5072

In [27]:
ship_data['x_G']*ship_data['scale_factor']/ship_data_wPCC['scale_factor']

-0.23989669902912625

In [28]:
ship_data['B']

1.3

In [29]:
data_wPCC_prime = model_MDL.prime_system.prime(data_wPCC[model_MDL.states_str + model_MDL.control_keys + ['V']], U=data_wPCC['V'])
data_wPCC_7m = model.prime_system.unprime(data_wPCC_prime, U=data_wPCC['V']*np.sqrt(scaling))
data_wPCC_7m.index*=np.sqrt(scaling)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 data_wPCC_prime = model_MDL.prime_system.prime(data_wPCC[model_MDL.states_str + model_MD     │
│ ❱ 2 data_wPCC_7m = model.prime_system.unprime(data_wPCC_prime, U=data_wPCC['V']*np.sqrt(scal     │
│   3 data_wPCC_7m.index*=np.sqrt(scaling)                                                         │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'scaling' is not defined

In [ ]:
dataframes = {
    'MDL test scaled':data_wPCC_7m,
}
dataframes.update(field_data)
track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'])
plot(dataframes, keys=['psi','V','r','delta'], units=units, symbols=symbols_dict);